In [2]:
import numpy as np
import pandas as pd
from scipy import stats
import time
from evcouplings.align import Alignment # install this using 'pip install evcouplings'
import glob


In [3]:
# Read in list of isolates
phenotypes =  pd.read_csv("../input_data/master_table_resistance.csv")
isolate_order = list(phenotypes.Isolate)

/home/ag300/jupytervenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (5,6,9,10,11,14,19,22,24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Iterate through each fasta file, select only the positions with a high enough MAF
# Keep only the isolates with phenotype data, reorder the alignment to match isolate_order
# save a reduced fasta file
!mkdir fastas_reduced
path_to_fastas = "/n/data2/hms/dbmi/beamlab/annachang/focus_cnn/fasta_files/"
for f in glob.glob(f"{path_to_fastas}/*.fasta"):
    print(f)
    # Reads the alignment file
    aln = Alignment.from_file(open(f))
    
    # Impose a minor allele frequency threshold of 0.001 in order to keep computation tractable
    # NOTE: the order in which you do the MAF thresholding and then subsetting the isolates matters!
    # Here it's fine because the only isolates in my fasta files are the ones that have phenotypes
    indices_to_keep = np.where(aln.frequencies.max(axis=1) < .999)[0]
    subset_alignment = aln.select(indices_to_keep)
    
    print("original alignment shape", aln.matrix.shape)
    print("reduced alignment shape", subset_alignment.matrix.shape)
    # Cleanup giant variables
    del aln

    # Get a list of the ids we are keeping, in order
    
    # First, correct the ids in the alignment so that they match the table of phenotypes
    subset_alignment.ids = [x.split("/")[-1].split(".")[0] for x in subset_alignment.ids]
    subset_alignment.id_to_index = {x:idx for idx,x in enumerate(subset_alignment.ids)}
    
    # Get the indices that would correctly reorder the alignment to match isolate_order
    reorder_index = [
        subset_alignment.id_to_index[x] for x in isolate_order if x in subset_alignment.id_to_index
    ]
    
    # Reorder based on reorder_index
    subset_alignment.ids = list(np.array(subset_alignment.ids)[reorder_index])
    subset_alignment.matrix = subset_alignment.matrix[reorder_index, :]
    print("shape with isolates ordered", subset_alignment.matrix.shape)
    
    # Get the name of the fasta file for saving
    name = f.split("/")[-1].split("_")[0]
    
    # save the reduced file
    subset_alignment.write(open(f"fastas_reduced/{name}_reduced.fa", "w"))
    np.save(f"fastas_reduced/{name}_indices.npy", indices_to_keep)
    
# Save the isolate indices in order, just in case
pd.DataFrame(subset_alignment.ids).to_csv("isolate_indexes.csv")

/n/data2/hms/dbmi/beamlab/annachang/focus_cnn/fasta_files/oxyR-ahpC_20201206.fasta
original alignment shape (23051, 1354)
reduced alignment shape (23051, 26)
shape with isolates ordered (23040, 26)
/n/data2/hms/dbmi/beamlab/annachang/focus_cnn/fasta_files/acpM-kasA_20201206.fasta
original alignment shape (23051, 1679)
reduced alignment shape (23051, 18)
shape with isolates ordered (23040, 18)
/n/data2/hms/dbmi/beamlab/annachang/focus_cnn/fasta_files/gid_20201206.fasta
original alignment shape (23051, 1042)
reduced alignment shape (23051, 187)
shape with isolates ordered (23040, 187)
/n/data2/hms/dbmi/beamlab/annachang/focus_cnn/fasta_files/FabG1-inhA_20201206.fasta
original alignment shape (23051, 2594)
reduced alignment shape (23051, 38)
shape with isolates ordered (23040, 38)
/n/data2/hms/dbmi/beamlab/annachang/focus_cnn/fasta_files/panD_20201213.fasta
original alignment shape (23051, 2187)
reduced alignment shape (23051, 26)
shape with isolates ordered (23040, 26)
/n/data2/hms/dbmi/

### Encode the fasta files for learning

In [5]:
### If using one-hot, here's the functions

# Mapping to use for one-hot encoding
BASE_TO_COLUMN = {'A': 0, 'C': 1, 'T': 2, 'G': 3, '-': 4}

# Get one hot vector
def make_one_hot(matrix, BASE_TO_COLUMN=BASE_TO_COLUMN):
    n_bases = len(BASE_TO_COLUMN.keys()) 
    one_hot = np.zeros((matrix.shape[0], matrix.shape[1]* n_bases),dtype=np.int8)
    print("starting from shape", one_hot.shape)
    
    for idx in range(matrix.shape[0]):
        for jdx in range(matrix.shape[1]):
            one_hot[idx, jdx*n_bases + BASE_TO_COLUMN[matrix[idx,jdx]]] = 1 

    return one_hot


In [6]:
# #### Make combined matrix of all the positions we'll be learning on

files = ['fastas_reduced/rpoBC_reduced.fa',
 'fastas_reduced/rpsL_reduced.fa',
 'fastas_reduced/gyrBA_reduced.fa',
 'fastas_reduced/clpC_reduced.fa',
 'fastas_reduced/embCAB_reduced.fa',
 'fastas_reduced/rrs-rrl_reduced.fa',
 'fastas_reduced/pncA_reduced.fa',
 'fastas_reduced/panD_reduced.fa',
 'fastas_reduced/tlyA_reduced.fa',
 'fastas_reduced/FabG1-inhA_reduced.fa',
 'fastas_reduced/KatG_reduced.fa',
 'fastas_reduced/gid_reduced.fa',
 'fastas_reduced/eis_reduced.fa',
 'fastas_reduced/oxyR-ahpC_reduced.fa',
 'fastas_reduced/ethAR_reduced.fa',
 'fastas_reduced/aftB-ubiA_reduced.fa',
 'fastas_reduced/acpM-kasA_reduced.fa',
 'fastas_reduced/rpsA_reduced.fa']

# Compute the total number of sites in our model by summing the length of all the alignment
total_sites = 0
for f in files:
    print(f)
    aln = Alignment.from_file(open(f))
    total_sites += aln.L
print("total sites", total_sites)
total_seqs = aln.N

fastas_reduced/rpoBC_reduced.fa
fastas_reduced/rpsL_reduced.fa
fastas_reduced/gyrBA_reduced.fa
fastas_reduced/clpC_reduced.fa
fastas_reduced/embCAB_reduced.fa
fastas_reduced/rrs-rrl_reduced.fa
fastas_reduced/pncA_reduced.fa
fastas_reduced/panD_reduced.fa
fastas_reduced/tlyA_reduced.fa
fastas_reduced/FabG1-inhA_reduced.fa
fastas_reduced/KatG_reduced.fa
fastas_reduced/gid_reduced.fa
fastas_reduced/eis_reduced.fa
fastas_reduced/oxyR-ahpC_reduced.fa
fastas_reduced/ethAR_reduced.fa
fastas_reduced/aftB-ubiA_reduced.fa
fastas_reduced/acpM-kasA_reduced.fa
fastas_reduced/rpsA_reduced.fa
total sites 3011


In [7]:

# Matrix to store the data for learning
X = np.zeros((total_seqs, total_sites), dtype=np.int8)

current_index = 0

for f in files:
    #print(f)
    aln = Alignment.from_file(open(f), alphabet='-ACGT')
    
    # Tells you which character is the most frequent in each site
    who_is_max = np.argmax(aln.frequencies, axis=1)
    
    # Major allele is encoded as 0, minor allele(s) as 1
    major_minor = aln.matrix_mapped != who_is_max
    
    # Add the encoding to the X matrix
    X[:, current_index:(current_index + major_minor.shape[1])] = major_minor
    
    # Keep track of how many sites in X we have filled in
    current_index = current_index + major_minor.shape[1]

np.save("combined_X.npy", X)
print(X.shape)

(23040, 3011)


In [8]:
# Make a table of all of the sites in the model for later mapping
# Note that the sites listed here are indexed within each fasta file - NOT the MTB genome
total_sites = []
genes = []

for f in files:
    print(f)
    
    gene_name = f.split("fastas_reduced/")[-1].split("_")[0]
    
    numpy_file = f.split("reduced.fa")[0] + "indices.npy"

    sites = np.load(numpy_file)
    
    sites = sorted(list(sites))
    
    total_sites += list(sites)
    genes += [gene_name] * len(list(sites))

print(len(genes), len(total_sites))
    
df = pd.DataFrame({
    "locus": genes,
    "sites": total_sites,
})

df.to_csv("site_indices.csv")

fastas_reduced/rpoBC_reduced.fa
fastas_reduced/rpsL_reduced.fa
fastas_reduced/gyrBA_reduced.fa
fastas_reduced/clpC_reduced.fa
fastas_reduced/embCAB_reduced.fa
fastas_reduced/rrs-rrl_reduced.fa
fastas_reduced/pncA_reduced.fa
fastas_reduced/panD_reduced.fa
fastas_reduced/tlyA_reduced.fa
fastas_reduced/FabG1-inhA_reduced.fa
fastas_reduced/KatG_reduced.fa
fastas_reduced/gid_reduced.fa
fastas_reduced/eis_reduced.fa
fastas_reduced/oxyR-ahpC_reduced.fa
fastas_reduced/ethAR_reduced.fa
fastas_reduced/aftB-ubiA_reduced.fa
fastas_reduced/acpM-kasA_reduced.fa
fastas_reduced/rpsA_reduced.fa
3011 3011


In [4]:
## Create combined genotype phenotype table

phenotypes =  pd.read_csv("/n/data2/hms/dbmi/beamlab/annachang/focus_cnn/master_table_resistance.csv", index_col=0)

# Convert phenotypes to numeric
y_all_rs = phenotypes.fillna('-1')
y_all_rs = y_all_rs.astype(str)
resistance_categories = {'R': 0, 'S': 1, '-1.0': -1, '-1': -1}

y_all = y_all_rs.copy()
for key, val in resistance_categories.items():
    y_all[y_all_rs == key] = val

# Fill missing data with nans
y_all[y_all==-1] = np.nan
phenotypes=y_all

# read in X matrix and isolate indexes 
X_all = np.load("combined_X.npy")

# Create a dataframe of X's with one row per isolate
indices = pd.read_csv("isolate_indexes.csv", index_col=0)
X_df = pd.concat([indices, pd.DataFrame(X_all)], axis=1)

# Add column names corresponding to position
column_names = pd.read_csv("site_indices.csv", index_col=0)
colnames = ["isolate"] + [f"{x}_{y}" for x,y in zip(column_names.locus, column_names.sites)]
X_df.columns = colnames

# Merge with phenotypes and save
X_df = X_df.merge(phenotypes, right_on="Isolate", left_on="isolate", how="left")
print(X_df.shape)
X_df.to_csv("combined_geno_pheno_df.csv")

(23040, 3051)
